In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC

In [2]:
#download necessary nltkk resources
nltk.download("vader_lexicon")
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
#data tweet, label
labeled_data = [
    ("I love this product. It has exceeded my expectations in every way. The quality is outstanding, and it's incredibly user-friendly. I can't imagine my life without it now.", "positive"),
    ("This product is terrible. I regret purchasing it. The quality is subpar and it broke within a week. I wouldn't recommend it to anyone.", "negative"),
    ("The quality could be better. While the product has some good features, there's definitely room for improvement. I'm on the fence about whether I'd buy it again.", "neutral"),
    ("I'm absolutely thrilled with this product. It has transformed how I do things on a daily basis. The innovative design sets it apart from the competition.", "positive"),
    ("Don't waste your money on this. I had high hopes, but it turned out to be a disappointment. The functionality is limited, and it feels cheaply made.", "negative"),
    ("It's an okay product. Nothing too impressive, but it gets the job done. I wish it had more advanced features though.", "neutral"),
    ("From the moment I started using this product, I fell in love. The attention to detail is evident, and it's clear that the creators put a lot of thought into it. A definite win!", "positive"),
    ("This has been a frustrating experience. The product arrived late, and it didn't work as described. I'm seriously considering returning it for a refund.", "negative"),
    ("I'm not entirely sure how I feel about this product. On one hand, it has some cool aspects, but on the other hand, it has some flaws that are hard to ignore.", "neutral"),
    ("I can't get enough of this product. It simplifies my life and saves me so much time. I can tell that the developers really understand the users' needs.", "positive"),
    ("What a waste of money. I had high expectations, but they were crushed. The product feels outdated, and the customer support is unresponsive.", "negative"),
    ("It's a decent product, but it lacks that wow factor. The quality is decent, and it's fairly reliable, but I was hoping for more.", "neutral"),
    ("This product is a game changer! It has exceeded my wildest expectations. The features it offers are unparalleled, and it's worth every penny.", "positive"),
    ("I'm extremely disappointed in this product. It arrived damaged, and the performance is far from what was advertised. I won't be recommending this to anyone.", "negative"),
    ("I'm still on the fence about this product. There are aspects that I like, but there are also some significant drawbacks. I'm not sure if it's the right fit for me.", "neutral"),
    ("I can't express how much I adore this product. It has genuinely made my life so much easier. I rave about it to all my friends and family.", "positive"),
    ("Honestly, I expected much more from this product. It falls short in many areas, and the quality is just not up to par. I wouldn't buy it again.", "negative"),
    ("It's a product with potential, but it's not quite there. The concept is interesting, but the execution could be better. I'm cautiously optimistic about its future updates.", "neutral"),
    ("This product is a must-have! I'm blown away by how well it works. It's evident that the company put a lot of research and effort into creating a top-tier product.", "positive"),
    ("I'm really dissatisfied with this product. It promised a lot, but it didn't deliver. The performance is lackluster, and I'm considering returning it.", "negative"),
    ("I'm still trying to make up my mind about this product. It has its merits, but it also has its drawbacks. I'll need more time to decide how I truly feel.", "neutral"),
    ("Wow, this service is fantastic! The support team was so helpful and the features are top-notch. I'm a very satisfied customer.", "positive"),
    ("I'm really disappointed in this service. It promised a lot, but the performance is lackluster and the interface is confusing. Definitely not worth the price.", "negative"),
    ("The functionality of this app is decent, but the user interface could use some improvement. I'm still exploring its features to see if it meets my needs.", "neutral"),
    ("I can't believe how fast the shipping was for this product. It arrived in perfect condition and the packaging was great. I'm impressed!", "positive"),
    ("Don't waste your money on this product. It's a total rip-off. The description doesn't match what you actually get. I'm very unsatisfied.", "negative"),
    ("I've been using this software for a while now. It's reliable and gets the job done. However, there are a few glitches that can be annoying.", "neutral"),
    ("The customer service was a delight to work with. They went above and beyond to solve my issue and made me feel valued as a customer. Highly recommended!", "positive"),
    ("I had high hopes for this gadget, but it's just average. It functions alright, but there's nothing special about it. I'm a bit underwhelmed.", "neutral"),
    ("This restaurant exceeded my expectations. The food was amazing, the atmosphere was cozy, and the service was impeccable. I'll definitely be coming back!", "positive"),
    ("What a waste of time using this website. It's clunky and slow, and I can never find what I'm looking for. I'm frustrated and won't be using it again.", "negative"),
    ("I'm still on the fence about this subscription box. The items in the first shipment were okay, but I'm not sure if it's worth the monthly cost.", "neutral"),
    ("This book is a masterpiece. The storytelling is captivating, and I couldn't put it down until I finished. A must-read for any fiction lover!", "positive"),
    ("I encountered so many bugs while using this software. It constantly crashes and I lost a lot of unsaved work. This has been a frustrating experience.", "negative"),
    ("The build quality of this product is solid. It feels sturdy and durable, like it's going to last a long time. I'm very impressed.", "positive"),
    ("I had such high expectations for this movie, but it fell short. The plot was confusing and the characters felt one-dimensional. I'm disappointed.", "negative"),
    ("The user interface of this app is intuitive and clean. I was able to navigate it easily without any confusion. A well-designed application.", "positive"),
    ("I've been using this skincare product for a week now, and I can already see a noticeable difference in my skin. It's smoother and more radiant. I'm thrilled!", "positive"),
    ("Stay away from this company's customer support. I had an issue with my order and they were unresponsive and unhelpful. Terrible experience.", "negative"),
    ("I'm pleasantly surprised by this app. It offers a wide range of features that I didn't expect to find. It's definitely worth the download.", "positive"),
    ("The sound quality of these headphones is subpar. The bass is muddy and the highs are distorted. I'm considering returning them.", "negative"),
    ("I purchased this product with low expectations, but it actually turned out to be quite useful. It's a good value for the price.", "positive"),
    ("I'm a bit conflicted about this game. The graphics are stunning, but the gameplay is repetitive. I'm not sure if I'll keep playing.", "neutral"),
    ("This clothing brand never disappoints. The fabric, stitching, and overall design of their clothes are consistently excellent. A brand I trust.", "positive"),
    ("Total waste of money. I ordered this gadget online and it arrived broken. The company's return process is a nightmare too. Frustrating experience.", "negative"),
    ("I'm still getting the hang of using this software. It has a steep learning curve, but I can see its potential once I master it.", "neutral"),
    ("I'm in love with this new restaurant. The fusion of flavors in their dishes is an adventure for the taste buds. Can't wait to go back!", "positive"),
    ("The battery life of this smartphone is disappointing. I have to charge it multiple times a day, even with minimal usage. Not practical at all.", "negative"),
    ("I'm thoroughly impressed by the performance of this vacuum cleaner. It picks up dirt and pet hair effortlessly, making cleaning a breeze.", "positive"),
    ("The movie had an intriguing premise, but it fell apart in the second half. The plot twists were forced and it left me unsatisfied. Not recommended.", "negative"),
    ("This fitness tracker helps me stay motivated. Seeing my daily progress has encouraged me to be more active. A great addition to my routine.", "positive"),
    ("The build quality of this toy is disappointing. It feels cheap and flimsy, and I'm worried it won't last long. Not what I was expecting.", "negative"),
    ("I'm on the fence about this e-book. The content is interesting, but the formatting is off, making it hard to read on some devices.", "neutral"),
    ("I can't believe how fast this website's interface is. It loads instantly and navigating through the pages is smooth. Impressive work!", "positive"),
    ("I regret upgrading to the premium version of this app. The additional features are underwhelming and not worth the extra cost. Disappointing.", "negative"),
    ("I have mixed feelings about this product. The design is sleek, but I've experienced some technical issues that are frustrating. Needs improvement.", "neutral"),
    ("The customer service of this online store is exceptional. They resolved my issue quickly and were very polite throughout the process. Highly recommended!", "positive"),
    ("I was excited to try out this recipe, but it turned out to be a disaster. The instructions were unclear, and the dish was inedible. Waste of ingredients.", "negative"),
    ("This software has saved me so much time and effort. The automation features are a game-changer for my work. I'm extremely satisfied.", "positive"),
    ("I'm on the fence about recommending this book. The story is engaging, but the writing style doesn't resonate with me. It might be a hit for others.", "neutral"),
    ("The sound quality of these speakers blew me away. The bass is deep, and the vocals are crystal clear. I'm enjoying my music like never before!", "positive"),
    ("Don't bother with this restaurant. The food was bland, and the service was slow. I won't be dining here again. Disappointed.", "negative"),
    ("I'm impressed by the variety of features in this productivity app. It has helped me stay organized and on top of my tasks. A valuable tool.", "positive"),
    ("This car rental service is a nightmare. The vehicle I received was dirty and had maintenance issues. They ruined my trip. Avoid at all costs.", "negative"),
    ("I'm amazed by the attention to detail in this video game. The world-building and character development are outstanding. A true masterpiece.", "positive"),
    ("I had high hopes for this fitness equipment, but it's not living up to expectations. The resistance levels are inconsistent, making workouts frustrating.", "negative"),
    ("This movie is a hidden gem. The plot is unique, and the actors delivered exceptional performances. A must-watch for cinephiles.", "positive"),
    ("The packaging of this product was excessive and wasteful. It's disappointing to see such disregard for the environment. Needs more sustainable options.", "negative"),
    ("I'm still undecided about this subscription service. The initial box was great, but the subsequent ones have been hit or miss. Jury's still out.", "neutral")

]

In [4]:
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()


In [5]:
preprocessed_data = []
labels = []

In [6]:
for tweet, label in labeled_data:
    tokens = word_tokenize(tweet.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    preprocessed_tweet = " ".join(lemmatized_tokens)
    
    preprocessed_data.append(preprocessed_tweet)
    labels.append(label)

In [7]:
#Split the preprocessed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data,labels,test_size = 0.2,random_state = 42)

In [8]:
# Vectorize the preprocessed data using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train)
X_test_vectors = vectorizer.transform(X_test)

In [9]:
# Train a Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel = "linear")
svm_classifier.fit(X_train_vectors, y_train)

SVC(kernel='linear')

In [10]:
# Evaluate the trained classifier on the testing set
y_pred = svm_classifier.predict(X_test_vectors)
classification_report = classification_report(y_test,y_pred)
print("Classification Report: ")
print(classification_report)

Classification Report: 
              precision    recall  f1-score   support

    negative       1.00      0.83      0.91         6
     neutral       1.00      1.00      1.00         3
    positive       0.83      1.00      0.91         5

    accuracy                           0.93        14
   macro avg       0.94      0.94      0.94        14
weighted avg       0.94      0.93      0.93        14



In [11]:
# Sentiment analysis of new, unseen tweet using Vader Sentiment analyzer
unseen_tweets = [
    "This movie was so disappointing. I had high hopes, but it turned out to be a complete letdown.",
    "Just tried the new restaurant in town and it was amazing! The food was delicious and the service was top-notch.",
    "I can't believe how poorly made this product is. It broke after just a few uses.",
    "I'm in love with the new skincare routine I started. My skin has never looked better!",
    "The customer support of this company is a nightmare. They never respond and don't seem to care about their customers.",
    "Attended the concert last night and it was an unforgettable experience. The energy of the crowd was electric!",
    "I regret buying this gadget. It's not user-friendly at all and the instructions are confusing.",
    "This book was a real page-turner. I couldn't put it down until I finished it!",
    "The traffic during rush hour is unbearable. I can't stand being stuck in my car for hours every day.",
    "I'm so impressed with the quality of the clothes from this brand. Definitely going to buy more!",
    "The battery life on this new phone is abysmal. I have to charge it multiple times a day.",
    "Spent the day at the amusement park and had a blast! The roller coasters were thrilling and the food was great.",
    "I had such high expectations for this event, but it fell flat. The organization was chaotic and there were many issues.",
    "Just watched the latest episode of my favorite TV show. It was so intense and left me eagerly waiting for the next one.",
    "The build quality of this furniture is impressive. It feels sturdy and well-made.",
    "I'm so frustrated with the constant software bugs on this app. It's practically unusable.",
    "This hiking trail was a hidden gem. The views were breathtaking and the hike was challenging in a good way.",
    "I ordered a dress online and it's nothing like the picture. The material is cheap and the sizing is way off.",
    "The service at this cafe was terrible. The staff was rude and inattentive throughout my visit.",
    "Just started learning a new instrument and it's been a struggle. Progress is slow, but I'm determined to improve.",
    "The hotel stay was wonderful. The room was comfortable and the staff went above and beyond to make me feel welcome.",
    "I can't believe how much money I wasted on this gym membership. I never have the motivation to go.",
    "The new software update is fantastic. It brings a lot of new features that I've been looking forward to.",
    "The customer service representative I spoke to was incredibly helpful and resolved my issue quickly.",
    "This movie was a pleasant surprise. I didn't expect much, but it turned out to be quite entertaining.",
    "The internet connection has been unreliable lately. It's frustrating when I'm trying to get work done."
]


In [12]:
analyzer = SentimentIntensityAnalyzer()
for tweet in unseen_tweets:
    sentiment_scores = analyzer.polarity_scores(tweet)
    print(f"Tweet: {tweet}")
    print(f"Sentiment Scores: {sentiment_scores}")
    print()

Tweet: This movie was so disappointing. I had high hopes, but it turned out to be a complete letdown.
Sentiment Scores: {'neg': 0.139, 'neu': 0.758, 'pos': 0.103, 'compound': -0.1675}

Tweet: Just tried the new restaurant in town and it was amazing! The food was delicious and the service was top-notch.
Sentiment Scores: {'neg': 0.0, 'neu': 0.698, 'pos': 0.302, 'compound': 0.8313}

Tweet: I can't believe how poorly made this product is. It broke after just a few uses.
Sentiment Scores: {'neg': 0.177, 'neu': 0.823, 'pos': 0.0, 'compound': -0.4215}

Tweet: I'm in love with the new skincare routine I started. My skin has never looked better!
Sentiment Scores: {'neg': 0.121, 'neu': 0.653, 'pos': 0.226, 'compound': 0.4742}

Tweet: The customer support of this company is a nightmare. They never respond and don't seem to care about their customers.
Sentiment Scores: {'neg': 0.118, 'neu': 0.761, 'pos': 0.121, 'compound': 0.0186}

Tweet: Attended the concert last night and it was an unforgettabl

In [13]:

analyzer = SentimentIntensityAnalyzer()

threshold_positive = 0.1
threshold_negative = -0.1

for tweet in unseen_tweets:
    sentiment_scores = analyzer.polarity_scores(tweet)
    compound_score = sentiment_scores['compound']
    
    if compound_score > threshold_positive:
        label = "positive"
    elif compound_score < threshold_negative:
        label = "negative"
    else:
        label = "neutral"
    
    print(f"Tweet: {tweet}")
    print(f"Predicted Label: {label}")
    print()


Tweet: This movie was so disappointing. I had high hopes, but it turned out to be a complete letdown.
Predicted Label: negative

Tweet: Just tried the new restaurant in town and it was amazing! The food was delicious and the service was top-notch.
Predicted Label: positive

Tweet: I can't believe how poorly made this product is. It broke after just a few uses.
Predicted Label: negative

Tweet: I'm in love with the new skincare routine I started. My skin has never looked better!
Predicted Label: positive

Tweet: The customer support of this company is a nightmare. They never respond and don't seem to care about their customers.
Predicted Label: neutral

Tweet: Attended the concert last night and it was an unforgettable experience. The energy of the crowd was electric!
Predicted Label: positive

Tweet: I regret buying this gadget. It's not user-friendly at all and the instructions are confusing.
Predicted Label: negative

Tweet: This book was a real page-turner. I couldn't put it down un